In [25]:
import numpy as np
from perceptronac.utils import read_im2bw
from perceptronac.losses import perfect_AC
from perceptronac.models import StaticAC
from perceptronac.models import CABAC
from perceptronac.coding2d import causal_context
from causal_context_2d import context_training as context_training_2

In [26]:
def get_loss_and_table(im,N):

    y,X = causal_context((im > 0).astype(int), N)
    cabac = CABAC(27)
    cabac.load(X=X,y=y)
    table = context_training_2(X,y,27)
    cabac_pred = cabac(X)
    final_loss = perfect_AC(y,cabac_pred,binary=True)
    
    return final_loss, table

In [2]:
file_name = "/home/lucas/Documents/data/TIP2017/ieee_tip2017_klt1024_3.png"
im = read_im2bw(file_name,0.4)

In [6]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(1,1,1)
ax.imshow(1-im, cmap='Greys')
ax.figure.savefig("page3TIP2017binary.png", dpi=300)

In [31]:
Ns = [0,2,4,10,26]
unseen_contexts = []
losses = []
table_sizes = []
for N in Ns:
    final_loss, table = get_loss_and_table(im,N)
    losses.append(final_loss)
    table_sizes.append(table.shape[0])
    unseen_contexts.append( np.count_nonzero(np.sum(table,axis=1)==0) )

[[0.]]
[[0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0.]
 [0. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0.]
 [0. 3. 2. 4. 0.]
 [0. 1. 0. 0. 0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  9.  6. 10.  0.  0.  0.]
 [ 0.  0.  7.  3.  2.  4.  8.  0.  0.]
 [ 0.  0.  5.  1.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. 24.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 21. 17. 14. 18. 22.  0.  0.  0.  0.]
 [ 0.  0.  0. 19. 11.  9.  6. 10. 12. 20.  0.  0.  0.]
 [ 0.  0. 25. 15.  7.  3.  2.  4.  8. 16. 26.  0.  0.]
 [ 0.  0. 23. 13.  5.  1.  0.  0.  0.  0.  0.  0.  0.]]


In [40]:
import pandas as pd
df = pd.DataFrame(
    data=zip(Ns,unseen_contexts,losses,table_sizes),columns=["N","dilution","bpp","contexts"]).set_index("N")
print(df.to_string())

    dilution       bpp  contexts
N                               
0          0  0.317674         1
2          0  0.197505         4
4          0  0.167735        16
10       188  0.141091      1024
26  67080469  0.067242  67108864
